# Tutorial 82: Tables

This example demonstrates how view, create, delete and edit measured variable tables (Sollwerttabellen).

# SIR 3S Installation

In [69]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S\SirGraf-90-15-00-21_Quebec-Upd2" #change to local path

# Imports

In [70]:
from sir3stoolkit.core import wrapper

In [71]:
from sir3stoolkit.mantle import mantle

The wrapper package has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [72]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Additional imports

In [73]:
import pandas as pd
import numpy as np


...

# Initialization

In [74]:
s3s = mantle.SIR3S_Model_Mantle()

Initialization complete


# Open Model

In [75]:
dbFilePath=r"Toolkit_Tutorial82_Model.db3"

In [76]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


# Create new measured variable tables

Let's insert Variable 1 as a placeholder example.

In [77]:
s3s.GetPropertiesofElementType(ElementType=s3s.ObjectTypes.MeasuredVariableTable)

['Name',
 'Beschreibung',
 'Zeitoption',
 'Intpol',
 'Idreferenz',
 'Pk',
 'InVariant']

In [78]:
measured_variable_table_1 = s3s.InsertElement(ElementType=s3s.ObjectTypes.MeasuredVariableTable, IdRef="1")

Element inserted successfully into the model with Tk: 5556095326232269080


In [79]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Name", Value="Variable 1")

Value is set


In [80]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Zeitoption", Value="0") # We do not want special time reference, see general SIR 3S documentation for details

Value is set


In [81]:
s3s.SetValue(Tk=measured_variable_table_1, propertyName="Intpol", Value="0") # We choose linear interpolation scheme. Use 9 for step function

Value is set


## Insert values

In [82]:
s3s.GetPropertiesofElementType(ElementType=s3s.ObjectTypes.MeasuredVariableTable_Row)

['Name', 'Fk', 'Zeit', 'Zeitstempel', 'W', 'Pk', 'InVariant']

W is the value that is given as a function of time W = f(t)

The sample data we provide is in 'measured' in constant time intervals and the values are random between 0 and 100.

In [83]:
# Create time vector: 0 to 600 seconds, step 20
time = np.arange(0, 601, 20)

# Create variable values between 0 and 100
# For reproducible randomness:
np.random.seed(42)
variable1 = np.random.randint(0, 101, size=len(time))

# Build the DataFrame
df = pd.DataFrame({
    "Time [s]": time,
    "Variable 1 [?]": variable1
})

In [84]:
df.head()

,Time [s],Variable 1 [?]
0,0,51
1,20,92
2,40,14
3,60,71
4,80,60


In [85]:
for id, row in df.iterrows():
    t = row['Time [s]']
    v = row['Variable 1 [?]']
    tk_new_row = s3s.AddTableRow(tablePkTk=measured_variable_table_1)[0]
    s3s.SetValue(Tk=tk_new_row, propertyName="Zeit", Value=str(t))
    s3s.SetValue(Tk=tk_new_row, propertyName="W", Value=str(v))

Row is added to the table with Tk: 5605805671787936164
Value is set
Value is set
Row is added to the table with Tk: 4830041275781582429
Value is set
Value is set
Row is added to the table with Tk: 5035519669336724198
Value is set
Value is set
Row is added to the table with Tk: 5161577548136955601
Value is set
Value is set
Row is added to the table with Tk: 5724893327685092194
Value is set
Value is set
Row is added to the table with Tk: 4923681690310231574
Value is set
Value is set
Row is added to the table with Tk: 4956262252197347867
Value is set
Value is set
Row is added to the table with Tk: 5451319129873794667
Value is set
Value is set
Row is added to the table with Tk: 5314723006334780533
Value is set
Value is set
Row is added to the table with Tk: 5125858064995569359
Value is set
Value is set
Row is added to the table with Tk: 4884168479466324455
Value is set
Value is set
Row is added to the table with Tk: 5136878820999409148
Value is set
Value is set
Row is added to the table wi

# View measured value tables

## Create dfs of values

In [86]:
table_tks = []

In [87]:
for id, tk in enumerate(s3s.GetTksofElementType(s3s.ObjectTypes.MeasuredVariableTable)):
    table_tks.append(tk)

In [88]:
table_dfs = []

In [89]:
for tk in table_tks:    
    rows = s3s.GetTableRows(tablePkTk=tk)
    row_tks = list(rows[0])  
    properties_rows_of_interest = ['Zeit', 'W']
    data = {
        tk: {prop: s3s.GetValue(tk, prop)[0] for prop in properties_rows_of_interest}
        for tk in row_tks
    }
    if data == {}:
        df = pd.DataFrame()
    else:
        df = pd.DataFrame.from_dict(data, orient='index')
    table_dfs.append(df)

## View value dfs and other data per measured variable table

In [90]:
def view_non_value_data_of_measured_variable_table(measured_variable_table_tk):
    name = s3s.GetValue(Tk=measured_variable_table_tk, propertyName="Name")[0]
    intpol = s3s.GetValue(Tk=measured_variable_table_tk, propertyName="Intpol")[0]
    if intpol == "0":
        intpol_clear = "linear"
    elif intpol == "9":
        intpol_clear = "step function"
    else:
        intpol_clear = "invalid intpol"
    print(f"Tk: {measured_variable_table_tk}, Name: {name}, Interpolation scheme: {intpol_clear}")

In [91]:
len(table_tks)

3

We have 3 tables to view.

### 1

In [92]:
view_non_value_data_of_measured_variable_table(table_tks[0])

Tk: 4690496339163833971, Name: Variable 2, Interpolation scheme: step function


In [93]:
table_dfs[0].head(10)

,Zeit,W
5238119681362566577,0,1
5061575663051528766,18,1
5723804092965624436,22,0
5048369715933631343,39,1
4615214940344953271,52,0
5375069532035517868,62,1
5339268850669424190,64,0
4965172431638973451,74,0
5205745981304652816,81,1
5367678067024287502,84,1


### 2

In [94]:
view_non_value_data_of_measured_variable_table(table_tks[1])

Tk: 4775965132419647289, Name: Variable 2, Interpolation scheme: step function


In [95]:
table_dfs[1].head(10)

,Zeit,W
5015901035241501518,0,99999


The second table obtained, does not contain any valid values, furthermore it carries the same name as the first one. So it probably is some duplicate relict from model creation. We can delete it afterwards.

In [96]:
table_tk_to_delete = table_tks[1]

### 3

In [97]:
view_non_value_data_of_measured_variable_table(table_tks[2])

Tk: 5556095326232269080, Name: Variable 1, Interpolation scheme: linear


In [98]:
table_dfs[2].head(10)

,Zeit,W
5605805671787936164,0,51
4830041275781582429,20,92
5035519669336724198,40,14
5161577548136955601,60,71
5724893327685092194,80,60
4923681690310231574,100,20
4956262252197347867,120,82
5451319129873794667,140,86
5314723006334780533,160,74
5125858064995569359,180,74


This is the table we created earlier, as can be seen, it worked out fine.

# Delete measured variable table

We can delete the faulty duplicate measured variable table as any other element. The rows depending on it will be deleted as well.

In [99]:
s3s.DeleteElement(Tk=str(table_tk_to_delete))

Element deleted successfully


# WORK IN PROGRESS: Joint view of all measured variable tables

The main challenge when trying to display multiple measured variable tablese is that the can have different time intervals. Many are measured in the same intervals, but some may only be 

In [100]:
s3s.CloseModel(False)

True